# Before you start:
This notebook shows how to execute YOLOv7 to detect loading bay door positions in video. The data for this project
was annotated using RoboFlow. 

It can be accessed at the following link:
https://app.roboflow.com/james-skelton/loading-dock/overview

Miscellaneous notes:
- Go to "Train" to train a model from scratch, "Test" to assess the quality of a trained model, and "Detect" to run detection on a sample. 
- To access the training data & pretrained model for this repo, run the following cell. You will then need to navigate to the "Helpers" section, and follow the instructions there to set up your code for YOLOv7. 

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [2]:
!pip install -r requirements.txt
!pip install setuptools==59.5.0
!pip install torchvision==0.11.3+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/protocol.py", line 36

## Helpers

To set up your RoboFlow annotated data for YOLO, we've provided some helper functions to quickly clean up the filenames. 

Simply run all the cells below

First, we create the directories for our data and download it. 

Second, we rename each of the files (they all start out in training directory) so they can be used with YOLO

Third, we move garagec001a-garage0440c to test. This will be our holdout to confirm the model is working. 

Fourth, we move garagec0448a - garagec0463c to valid, and garagee0001a-garagee0150c to validation.
This will be the set of images the model actively tests itself on. and these losses are used to improve the model step-to-step

In [2]:
# we create the directories for our data and download it.
!mkdir v-test
!mkdir test
!mkdir valid

In [3]:
! cd v-test
!curl -L "https://app.roboflow.com/ds/jUIVg1VfEY?key=ZUmMgxskv6" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
! cd ..

In [ ]:
import os 


# remove roboflow extra junk to clean up file names for Yolov7

count = 0
for i in sorted(os.listdir('v-test/train/labels')):
    if count >=3:
        count = 0
    count += 1
    # print(count)
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    # print(j[0]+'.txt')
    source = 'v-test/train/labels/'+i
    dest = 'v-test/train/labels/'+j[0]+dict1[count]+'.txt'
    # print(source)
    # print(dest)
    # break
    os.rename(source, dest)
# remove roboflow extra junk
count = 0
for i in sorted(os.listdir('v-test/train/images')):
    if count >=3:
        count = 0
    count += 1
    # print(count)
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    # print(j[0]+'.txt')
    source = 'v-test/train/images/'+i
    dest = 'v-test/train/images/'+j[0]+dict1[count]+'.jpg'
    # print(source)
    # print(dest)
    # break
    os.rename(source, dest)

In [2]:
# Set up validation set

import os
import shutil
 
source = 'v-test/train/images/'
destination = 'v-test/valid/images/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) >=463 and int(f[7:-5]) <=467:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('ee')[0] == 'garag' and int(f[7:-5]) <= 150:
        # print(int(f[7:-5]))
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
 
source = 'v-test/train/labels/'
destination = 'v-test/valid/labels/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) >=463 and int(f[7:-5]) <=467:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('ee')[0] == 'garag' and int(f[7:-5]) <= 150:
        # print(int(f[7:-5]))
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

In [9]:
import os
import shutil
 
source = 'v-test/train/images/'
destination = 'v-test/test/images/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) <=440:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

source = 'v-test/train/labels/'
destination = 'v-test/test/labels/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) <=440:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

## Train


This section shows how to fine tune a model for a custom dataset.

### Training instructions & tips
- If you need to change the locations of your training/validation/test images, then be sure to go to custom.yaml in the "data" folder, and change the path locations. 
- Also in coco.yaml, you can set and label the number of labels you want to be able to detect with your model
    - We are using 4 labels: open, close, truck-open, and truck close. These can be changed in coco.yaml to reflect the task at hand
- If you are on a distributed machine, use the second train command in the cell below. Be sure to change "nproc_per_node" to accurately reflect the number of GPUs on your device. 
- Use the hyp.scratch.custom.yaml file to change hyperparameters for training

In [9]:
# !python train.py --workers 8 --device 0 --batch-size 32 --data data/coco.yaml --img 416 416 --cfg cfg/training/yolov7x.yaml --weights runs/train/yolov7-custom/weights/best.pt --name yolov7-custom2 --hyp data/hyp.scratch.custom.yaml --epochs 100

!python -m torch.distributed.launch --nproc_per_node 2 --master_port 9527 train.py --workers 16 --device 0,1 --sync-bn --batch-size 16 --data data/coco.yaml --img 1280 720 --cfg cfg/training/yolov7.yaml --weights yolov7_training.pt --name yolov7-loading-door --hyp data/hyp.scratch.custom.yaml --epochs 200


/usr/local/lib/python3.9/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
YOLOR 🚀 v0.1-51-g2596994 torch 1.10.2+cu111 CUDA:0 (RTX A6000, 48685.375MB)
                                            CUDA:1 (RTX A6000, 48685.375MB)

Added key: store_based_barrier_key:1 to store for rank: 0
Rank 0: Completed store-b

## Detect

Use the following cell to run detection on a submitted image. 

- Change image or video being detected on using --source tag
- img size X dimension must be correct for this to run. no Y needed
- The designated test set can be found at v-test/test/ or, in video form with borders, at test-final-video.mp4

In [8]:
!python detect.py --weights runs/train/yolov7-loadingdock18/weights/best.pt --conf 0.25 --img-size 1920 --source test-video-final.mp4 --name test


Namespace(weights=['runs/train/yolov7-loadingdock18/weights/best.pt'], source='test-video-final.mp4', img_size=1920, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='test', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-51-g2596994 torch 1.10.2+cu111 CUDA:0 (RTX A6000, 48685.375MB)
                                            CUDA:1 (RTX A6000, 48685.375MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36497954 parameters, 6194944 gradients, 103.

## Testing

YOLOv7 comes with a built in testing function. It will test our model on the validation set normally. I have also provided data/test.yaml, so you can switch the --data flag to it if you want to test on the test data.

In [87]:
!python test.py --data data/coco.yaml --img 1280 --batch 32 --conf 0.001 --iou 0.65 --device 0 --weights runs/train/yolov7-loadingdock15/weights/last.pt --name yolov7_1280_testing


Namespace(weights=['runs/train/yolov7-loadingdock15/weights/last.pt'], data='data/coco.yaml', batch_size=32, img_size=1280, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='yolov7_1280_testing', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-51-g2596994 torch 1.10.2+cu111 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36497954 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 t